<a href="https://colab.research.google.com/github/PabloGuev17/ProcesamientoDatosMasivos/blob/main/Proceso_de_b%C3%BAsqueda_de_elementos_similares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# El proceso de búsqueda de elementos similares

En este tutorial vamos a repasar todo el proceso de búsqueda de elementos similares, con el objetivo de buscar, de un conjunto de 10 textos de poesías de Pablo Neruda, cuales son los dos más similares.

Destacamos las siguientes etapas.
- Shingling. Esto es, transformar el texto a conjuntos. Ya lo hemos demostrado en otro tutorial, pero dejamos el código por continuidad.
- Minhashing. Para poder calcular la similitud, computaremos una firma de Minhash de tamaño 5 para cada uno de los conjuntos, en este caso, cada uno de los shingles de los textos.
- Comparación de firmas de Minhash para estimar la similitud de Jaccard. Cuando tenemos la firma de minhash para cada texto, nos concentramos en comparar sus firmas de MinHash. Entonces, compararemos con la similitud de Jaccard solo aquellos textos que presenten la misma firma de MinHash.

En este tutorial se repasa el proceso de busqueda de elementos similares, buscar cuales son los 2 elementos mas silimiares, usando la tecnica de minHash.

1era etapa: Shigling donde se transforma el texto a conjunto
2da etapa: MinHashing para poder calcular la similitud.
3era Etapa: Comparación de firmas de MinHash para estimar la similutd de Jaccard.

## Shingling

La técnica de *k*-shingling consiste en transformar un texto (es decir, un string) a un conjunto formado por todos los substring de tamaño *k* de ese texto, incluyendo espacios y otros caracteres no lexicográficos.

Veamos como hacer esto para un conjunto de 10 poesías escritas por Pablo Neruda, parte de su obra de Odas Elementales. *italicized text*

In [1]:
### Para almacenar los distintos textos creamos un diccionario.
odas = {}

### Vamos a subir cada texto indexado por un keyword distinto en este diccionario.
### Para hacerlos más legibles, reemplazamos los fin de línea por un espacio.

with open("oda_alegria.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['alegria']=text

with open("oda_caldillo.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['caldillo']=text

with open("oda_feliz.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['feliz']=text

with open("oda_libro.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['libro']=text

with open("oda_mar.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['mar']=text

with open("oda_poetas.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['poetas']=text

with open("oda_tiempo.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['tiempo']=text

with open("oda_tristeza.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['tristeza']=text


with open("oda_valparaiso.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['valparaiso']=text


with open("oda_vino.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['vino']=text

### Como vemos, el resultado es que cada uno de estos textos es un string,
### indexado en el diccionario por un keyword diferente.

print(odas)

{'alegria': 'Alegría hoja verde caída en la ventana, minúscula claridad recién nacida, elefante sonoro, deslumbrante moneda, a veces ráfaga quebradiza, pero más bien pan permanente, esperanza cumplida, deber desarrollado. Te desdeñé, alegría. Fui mal aconsejado. La luna me llevó por sus caminos. Los antiguos poetas me prestaron anteojos y junto a cada cosa un nimbo oscuro puse, sobre la flor una corona negra, sobre la boca amada un triste beso. Aún es temprano. Déjame arrepentirme. Pensé que solamente si quemaba mi corazón la zarza del tormento, si mojaba la lluvia mi vestido en la comarca cárdena del luto, si cerraba los ojos a la rosa y tocaba la herida, si compartía todos los dolores, yo ayudaba a los hombres. No fui justo. Equivoqué mis pasos y hoy te llamo, alegría.  Como la tierra eres necesaria.  Como el fuego sustentas los hogares.  Como el pan eres pura.  Como el agua de un río eres sonora.  Como una abeja repartes miel volando.  Alegría, fui un joven taciturno, hallé tu cabel

In [2]:
### El siguiente pedazo de código genera un diccionario, indexado bajo los mismos keywords que los textos,
### En este diccionario almacenamos el resultado de hacer k-shingling en el texto.

### Probemos con k = 3

odas_shingles = {}
k = 3

### iteramos sobre todas las odas
for (name,text) in odas.items():

    ### Es importante declarar los shingles como un set (conjunto), de forma que no hayan duplicados
    odas_shingles[name] = set()

    ### y nos concentramos en todos los substrings entre las posiciones i y la i+k+1, para un
    ### i que parte desde el inicio del texto

    for i in range(len(text) - k-1):
        shingle = text[i:i+k]
        odas_shingles[name].add(shingle)


In [3]:
### Este es el resultado de hacer shingling al primer texto, la Oda a la Alegría.

print(odas_shingles['alegria'])

{'res', 'yud', 'ejo', 'ebl', ' is', 'edr', 'ces', 'tó ', 'ra,', ' va', 'dic', ' so', 'qui', 'pli', 'sol', 'n j', 'e. ', 'tac', ' am', 'n t', 'en ', 'aro', 'ped', ' me', 'os.', 'son', 'nta', 'ibi', 'do ', 'pic', ' ra', 'ona', 'grí', 'cur', 'cam', ' fu', 'las', ' fl', 'can', 'sa ', ' ni', 'a! ', 'Ale', 'ó s', 'Hoy', ' Co', ' ba', ' rí', 'leñ', 'aíd', 'nte', 'mar', 'ntr', 'ras', 'anz', 'don', 'ana', 'sla', 'e t', 'ria', '  H', 'No ', 'e q', 'ven', 'dal', ' ll', 'mal', 'ron', 'a d', 'abe', 'sti', 'la ', 'tra', 'bes', 'sto', 'a. ', 'goc', 'zad', 's v', 'rea', 'ach', 'sé ', 'í s', 'aga', 'ro,', 'ado', ', s', 'mpr', ' cá', 'her', 'ere', 'e u', 'los', 'ros', 'nad', 'evó', 'nto', 'adi', 'bel', 'o q', ' Fu', 'es.', ' bi', 'nde', ' Y ', 'bro', 'e: ', ' le', 'eri', 'o m', 'Los', 'is ', 'rtí', 'cer', 'ura', 'caí', 'enc', 'tes', 'apr', ' oj', 'ato', 'hom', 'erd', 'e l', 'ano', 'ueg', 'via', 'uos', 'i p', 'pob', 'y, ', 'min', 'e s', ' un', 'ngr', 'bo ', 'ue ', 'ndí', 'lut', 'ono', 'efa', 'cat', 'ari'

## Jaccard Similarity

Vamos a comparar cual de estas 10 odas es la más parecida entre sí, según la similitud de Jaccard.

In [4]:
### Lo primero es una funcion que calcula la similitud de Jaccard entre dos conjuntos.
### Recordemos que se define como la proporcion entre el tamaño de la intersección de los conjuntos,
### dividido por el tamaño de su unión.

def jaccard_similarity(set1, set2):
    # Computa la similitud de Jaccard entre dos sets
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

In [5]:
### Un par de pruebas:
### - La similitud de un conjunto con si mismo debe ser 1
### - La similitud de un conjunto con el vacio debe ser cero
### - Probamos cuanto es la similitud entre los primeros dos textos.

print(jaccard_similarity(odas_shingles['alegria'],odas_shingles['alegria']))
print(jaccard_similarity(odas_shingles['alegria'],{}))
print(jaccard_similarity(odas_shingles['alegria'],odas_shingles['caldillo']))

1.0
0.0
0.2990239574090506


Ahora vamos a calcular la similitud para todas las $10 \choose 2$ combinaciones.
El resultado irá a una matriz de 10x10, donde la entrada $[i][j]$ es el valor
de la similitud entre la i-ésima oda y la j-ésima oda.

In [6]:
### Armamos la matriz

n = len(odas_shingles)
similarity_matrix = [[0 for i in range(n)] for j in range(n)]

### con esto podemos usar names[i] para conseguir el nombre de la i-esima oda.
names = list(odas_shingles.keys())

### Llenamos la matriz
for i in range(n):
    for j in range(n):
        if i != j:
            similarity = jaccard_similarity(odas_shingles[names[i]], odas_shingles[names[j]])
            similarity_matrix[i][j] = similarity

for row in similarity_matrix:
    print(row)


[0, 0.2990239574090506, 0.2569778633301251, 0.32940185341196293, 0.3416726233023588, 0.34425036390101893, 0.3205357142857143, 0.2358036573628489, 0.3596138374899437, 0.3537190082644628]
[0.2990239574090506, 0, 0.2616136919315403, 0.2898120672601385, 0.309640522875817, 0.3119266055045872, 0.3041125541125541, 0.23774509803921567, 0.3233644859813084, 0.33463796477495106]
[0.2569778633301251, 0.2616136919315403, 0, 0.24160346695557963, 0.27088830255057167, 0.2533215234720992, 0.28308823529411764, 0.23121387283236994, 0.2406311637080868, 0.25494276795005205]
[0.32940185341196293, 0.2898120672601385, 0.24160346695557963, 0, 0.3187403993855607, 0.3325434439178515, 0.3111332007952286, 0.2600896860986547, 0.33507853403141363, 0.3114463176574978]
[0.3416726233023588, 0.309640522875817, 0.27088830255057167, 0.3187403993855607, 0, 0.3537832310838446, 0.2985553772070626, 0.24171029668411867, 0.3649253731343284, 0.3575248281130634]
[0.34425036390101893, 0.3119266055045872, 0.2533215234720992, 0.3325

En este caso lo podemos hacer de forma sencilla, ya que son solo 10 textos pequeños. Cuando la cantidad de conjuntos es muy grande, probar para todas las combinaciones es muy costoso, y debemos usar MinHashing.

Para terminar, veamos cuales dos odas son las mas similares.

In [7]:
maximo = 0
for i in range(n):
    for j in range(n):
        valor = similarity_matrix[i][j]
        if maximo < valor:
            i_max = i
            j_max = j
            maximo = valor
print(i_max,j_max,maximo)

4 8 0.3649253731343284


In [8]:
names[4],names[8]

('mar', 'valparaiso')

## Minhashing

Para calcular las firmas de minhash, vamos a usar cuatro funciones de hash, las que intuitivamente jugarán el rol de una permutación de la matriz de elementos.
Veamos primero que funciones de hash podemos usar. Estas derivan de usar la operación módulo (%), y tienen la ventaja de que simulan permutaciones de la matriz de elementos - conjuntos: $h(i)$ nos dice en que orden está la fila $i$ en la permutación.

In [9]:
### Contamos la cantidad de elementos totales que tenemos,
### como el tamaño de la union de todos los conjuntos de shingles.

un = set()
for (name,shingle) in odas_shingles.items():
        un = un.union(shingle)

print(len(un))

2349


La matriz de elementos tiene, entonces, 2349 filas. Las permutaciones serán funciones que transformen números entre
0 y 2348 a numeros entre 0 y 2348. Definimos 5 de estas funciones, aunque podría ser cualquier función de hash.

In [10]:
def h0(n):
    return (n+4) % 2349
def h1(n):
    return (2*n+1) % 2349
def h2(n):
    return (2*n) % 2349
def h3(n):
    return (3*n+300) % 2349
def h4(n):
    return (n+1000) % 2349


Residuo = Dividendo -(Cociente Entero x Divisor)


In [11]:
### Veamos algunos ejemplos. En estas funciones, la antigua fila 1 sería la siguiente fila en la permutación:

print(h0(1),h1(1),h2(1),h3(1),h4(1))

5 3 2 303 1001


In [12]:
### Y la antigua fila 100 sería la siguiente fila en la permutación:

print(h0(100),h1(100),h2(100),h3(100),h4(100))

104 201 200 600 1100


### Calculando la firma de minhash de las odas

Como ya sabemos, la matriz de elementos - conjuntos tiene 2349 filas, y 10 columnas, una por cada oda. Vamos a reducir esto a una matriz de 5 filas y 10 columnas: para cada columna, representando una oda, entregamos la firma de minhash correspondiente a la función de hash hi.

In [13]:
### Creamos la matriz FH con las filas y columnas correspondientes.
### Inicialmente la matriz tiene solo valores 3000, cualquier valor mas grande
### que la cantidad de elementos sirve.

n = len(odas_shingles)
FH = [[3000 for j in range(n)] for i in range(5)]

### Tomamos la union de todos los elementos en el conjunto llamado un,
### y para iterar sobre estos, lo transformamos a una lista ordenada

for (name,shingle) in odas_shingles.items():
    un = un.union(shingle)
    sorted_union = sorted(un)

### con esto podemos usar names[i] para conseguir el nombre de la i-esima oda.
names = list(odas_shingles.keys())

             ### iteramos cada elemento
for e in range(0,len(sorted_union)):
    ### iteramos cada columna, correspondiente a la cantidad de textos (odas en nuestro caso)
    for i in range(0,n):
        ### si el elemento e no está en el texto i, no hacemos nada
        if sorted_union[e] not in odas_shingles[names[i]]:
            pass
        ### si el elemento si está, computamos los valores de h(e) y
        else:
            FH[0][i]=min(FH[0][i],h0(e))
            FH[1][i]=min(FH[1][i],h1(e))
            FH[2][i]=min(FH[2][i],h2(e))
            FH[3][i]=min(FH[3][i],h3(e))
            FH[4][i]=min(FH[4][i],h4(e))